In [1]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [2]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [3]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [4]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [37]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [38]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
#device = 'cpu'


device(type='cuda', index=0)

In [7]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.0510, 0.3781, 0.2525],
        [0.0787, 0.9913, 0.4952],
        [0.6603, 0.7436, 0.1571],
        [0.1668, 0.8830, 0.7608],
        [0.5752, 0.5757, 0.8556]], device='cuda:0')


In [8]:
x.get_device()

0

In [9]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [10]:
torch.cuda.memory_allocated()

0

In [42]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Sat Feb 18 01:55:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   31C    P0    35W / 250W |   1190MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [20]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
# Case2: codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000 for the model 'EleutherAI/gpt-neo-2.7B' <-- MEMORY CONSTRAINTS
# Case3: codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000 for the model 'EleutherAI/gpt-neo-1.3B'
# Case4: codesearch_tesbed_microsoft-CodeGPT-small-py_10000 for the model 'microsoft/CodeGPT-small-py-adaptedGPT2'
def params(): 
    return {
            'big_table_path' : '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py_10000.csv',
            'hf_model' : 'microsoft/CodeGPT-small-py-adaptedGPT2',
            'model_name': 'CodeGPT-small-py_10000_',
            'callbacks' : '../data/callbacks-CodeGPT-small-py_10000',
            'batch': 1 
}

In [21]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py_10000.csv'

# Data Upload

In [14]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [15]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,459.550200
std,583.826137
min,41.000000
25%,161.750000
50%,278.000000
75%,529.000000
max,16454.000000


In [16]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def find_function(self, context, funname):\n ...","[('def', 'def', 'function_definition'), ('find...","[(1336, 'def', 'function_definition'), (1766, ...","[1336, 1766, 65, 1590, 10, 270, 14, 1245, 14, ...",477
1,"def serialCmdPwdAuth(self, password_str):\n ...","[('def', 'def', 'function_definition'), ('seri...","[(1336, 'def', 'function_definition'), (2603, ...","[1336, 2603, 8644, 50, 2409, 3084, 10, 270, 14...",520
2,"def email(cls, invoice, kind):\n ''' Se...","[('def', 'def', 'function_definition'), ('emai...","[(1336, 'def', 'function_definition'), (3306, ...","[1336, 3306, 10, 1173, 14, 14699, 14, 4033, 29...",108
3,"def resolve(cursor, key):\n """"""\n Get en...","[('def', 'def', 'function_definition'), ('reso...","[(1336, 'def', 'function_definition'), (5179, ...","[1336, 5179, 10, 2882, 14, 570, 298, 201, 223,...",178
4,"def _get_new_csv_writers(trans_title, meta_tit...","[('def', 'def', 'function_definition'), ('_get...","[(1336, 'def', 'function_definition'), (440, '...","[1336, 440, 344, 65, 1073, 65, 3046, 65, 23630...",157


In [45]:
data_pd_to_compare = pd.read_csv( 
                      '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv' , 
                      index_col=0
            )

In [46]:
data_pd_to_compare.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def child_object(self):\n """""" Get Task ...","[('def', 'def', 'function_definition'), ('chil...","[(4299, 'def', 'function_definition'), (1200, ...","[4299, 1200, 62, 15252, 7, 944, 2599, 198, 220...",92
1,"def manual_run(turn_into_run=True, store_meta_...","[('def', 'def', 'function_definition'), ('manu...","[(4299, 'def', 'function_definition'), (10107,...","[4299, 10107, 62, 5143, 7, 15344, 62, 20424, 6...",484
2,"def connectionLost(self, reason):\n """"""...","[('def', 'def', 'function_definition'), ('conn...","[(4299, 'def', 'function_definition'), (4637, ...","[4299, 4637, 31042, 7, 944, 11, 1738, 2599, 19...",94
3,"def as_thing_description(self):\n """"""\n...","[('def', 'def', 'function_definition'), ('as_t...","[(4299, 'def', 'function_definition'), (355, '...","[4299, 355, 62, 1197, 62, 11213, 7, 944, 2599,...",1080
4,"def sample_uniform_initial_state(parameter,\n ...","[('def', 'def', 'function_definition'), ('samp...","[(4299, 'def', 'function_definition'), (6291, ...","[4299, 6291, 62, 403, 6933, 62, 36733, 62, 521...",570


# Model Upload

In [18]:
#! pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [19]:
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git


In [20]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")


def testing_1():
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [21]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [49]:
parameters['hf_model']

'microsoft/CodeGPT-small-py-adaptedGPT2'

In [50]:
#This code works for CodeGPT
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
#tokenizer.get_vocab()

In [23]:
#len( tokenizer.get_vocab() ) #todo an assert

In [51]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [52]:
len(prompts)

10000

In [47]:
prompts[0]

'def find_function(self, context, funname):\n        """Find a function in the given context by name.\n\n        This function will first search the list of builtins and if the\n        desired function is not a builtin, it will continue to search\n        the given context.\n\n        Args:\n            context (object): A dict or class that is a typedargs context\n            funname (str): The name of the function to find\n\n        Returns:\n            callable: The found function.\n        """\n\n        if funname in self.builtins:\n            return self.builtins[funname]\n\n        func = None\n        if isinstance(context, dict):\n            if funname in context:\n                func = context[funname]\n\n                #Allowed lazy loading of functions\n                if isinstance(func, str):\n                    func = self._deferred_add(func)\n                    context[funname] = func\n        elif hasattr(context, funname):\n            func = getattr(context, 

In [53]:
#Making space for the tensors
#input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor [cast to list]
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [54]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [55]:
input_ids_list[0]

tensor([ 4299,  1064,    62,  8818,     7,   944,    11,  4732,    11,  1257,
         3672,  2599,   198,   220,   220,   220,   220,   220,   220,   220,
        37227, 16742,   257,  2163,   287,   262,  1813,  4732,   416,  1438,
           13,   628,   220,   220,   220,   220,   220,   220,   220,   770,
         2163,   481,   717,  2989,   262,  1351,   286,  3170,  1040,   290,
          611,   262,   198,   220,   220,   220,   220,   220,   220,   220,
        10348,  2163,   318,   407,   257,  3170,   259,    11,   340,   481,
         2555,   284,  2989,   198,   220,   220,   220,   220,   220,   220,
          220,   262,  1813,  4732,    13,   628,   220,   220,   220,   220,
          220,   220,   220,   943, 14542,    25,   198,   220,   220,   220,
          220,   220,   220,   220,   220,   220,   220,   220,  4732,   357,
        15252,  2599,   317,  8633,   393,  1398,   326,   318,   257, 25683,
        22046,  4732,   198,   220,   220,   220,   220,   220, 

In [56]:
#It should be same size
print(len(input_ids_list),len(prompts))
assert len(input_ids_list) == len(prompts)

9726 10000


AssertionError: 

In [29]:
data_pd.model_input_ids.values[0]

'[4299, 37773, 62, 17566, 7, 18982, 62, 11600, 2599, 198, 220, 220, 220, 37227, 65, 2414, 12, 12685, 4147, 4263, 287, 257, 3359, 12984, 5794, 8633, 198, 220, 220, 220, 220, 198, 220, 220, 220, 8673, 428, 815, 307, 12118, 287, 33918, 62, 27773, 2346, 30, 198, 220, 220, 220, 220, 198, 220, 220, 220, 40117, 198, 220, 220, 220, 24200, 438, 198, 220, 220, 220, 220, 198, 220, 220, 220, 5794, 62, 11600, 1058, 8633, 198, 220, 220, 220, 220, 220, 220, 220, 317, 22155, 286, 3359, 1366, 1994, 276, 416, 285, 524, 12, 4906, 198, 220, 220, 220, 220, 198, 220, 220, 220, 16409, 198, 220, 220, 220, 35656, 198, 220, 220, 220, 220, 198, 220, 220, 220, 5794, 62, 11600, 1058, 8633, 198, 220, 220, 220, 220, 220, 220, 220, 317, 4866, 286, 262, 976, 22155, 11, 198, 220, 220, 220, 220, 220, 220, 220, 475, 13934, 2939, 1366, 19203, 9060, 14, 11134, 6, 393, 705, 9060, 14, 73, 22071, 11537, 198, 220, 220, 220, 220, 220, 220, 220, 318, 2779, 2414, 12, 12685, 9043, 13, 198, 220, 220, 220, 220, 198, 220, 220, 220, 3

In [30]:
input_ids_list[0]

tensor([ 4299, 37773,    62, 17566,     7, 18982,    62, 11600,  2599,   198,
          220,   220,   220, 37227,    65,  2414,    12, 12685,  4147,  4263,
          287,   257,  3359, 12984,  5794,  8633,   198,   220,   220,   220,
          220,   198,   220,   220,   220,  8673,   428,   815,   307, 12118,
          287, 33918,    62, 27773,  2346,    30,   198,   220,   220,   220,
          220,   198,   220,   220,   220, 40117,   198,   220,   220,   220,
        24200,   438,   198,   220,   220,   220,   220,   198,   220,   220,
          220,  5794,    62, 11600,  1058,  8633,   198,   220,   220,   220,
          220,   220,   220,   220,   317, 22155,   286,  3359,  1366,  1994,
          276,   416,   285,   524,    12,  4906,   198,   220,   220,   220,
          220,   198,   220,   220,   220, 16409,   198,   220,   220,   220,
        35656,   198,   220,   220,   220,   220,   198,   220,   220,   220,
         5794,    62, 11600,  1058,  8633,   198,   220,   220, 

In [31]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [32]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2560)
    (wpe): Embedding(2048, 2560)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
        )
        (ln_2): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2560, out_features=10240, bias=True)
          (c_proj): Linear

In [33]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv',
 'hf_model': 'EleutherAI/gpt-neo-2.7B',
 'model_name': 'EleutherAI-gpt-neo-2.7B_10000_',
 'callbacks': '../data/callbacks',
 'batch': 1}

In [34]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.type(torch.LongTensor).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
            

    pass

In [35]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [36]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [37]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [38]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [39]:
#output_loss

In [40]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=2)

Batch [2] Completed
Batch [3] Completed
Batch [4] Completed
Batch [5] Completed
Batch [6] Completed
Batch [7] Completed
Batch [8] Completed
Batch [9] Completed
Batch [10] Completed
Batch [11] Completed
Batch [12] Completed
Batch [13] Completed
Batch [14] Completed
Batch [15] Completed
Batch [16] Completed
Batch [17] Completed
Batch [18] Completed
Batch [19] Completed
Batch [20] Completed
Batch [21] Completed
Batch [22] Completed
Batch [23] Completed
Batch [24] Completed
Batch [25] Completed
Batch [26] Completed
Batch [27] Completed
Batch [28] Completed
Batch [29] Completed
Batch [30] Completed
